In [ ]:
from kafka import KafkaConsumer
import psycopg2
import json

consumer = KafkaConsumer(
    'error_log',
    bootstrap_servers = 'kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id = 'invalid-data',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

conn = psycopg2.connect(
    dbname='mydb',
    user='myuser',
    password='myuserpass',
    host='postgres',
    port=5432
)
cursor = conn.cursor()

for msg in consumer:
    data = msg.value
    print(data)
    cursor.execute(
        "INSERT INTO error_log (device_id, error_message, event_timestamp) VALUES (%s, %s, %s)",
        (data['device_id'],  data['error_message'], data['event_timestamp'])
    )
    conn.commit()
    print(f"Inserted into DB: {data}")


{'device_id': 'sensor-0', 'temperature': -9.86, 'humidity': 116.28, 'event_timestamp': 1747327889.3125596, 'error_message': '-9.86 is less than the minimum of 0'}
Inserted into DB: {'device_id': 'sensor-0', 'temperature': -9.86, 'humidity': 116.28, 'event_timestamp': 1747327889.3125596, 'error_message': '-9.86 is less than the minimum of 0'}
{'device_id': 'sensor-1', 'temperature': 59.65, 'humidity': 40.09, 'event_timestamp': 1747327889.8158782, 'error_message': '59.65 is greater than the maximum of 50'}
Inserted into DB: {'device_id': 'sensor-1', 'temperature': 59.65, 'humidity': 40.09, 'event_timestamp': 1747327889.8158782, 'error_message': '59.65 is greater than the maximum of 50'}
{'device_id': 'sensor-2', 'temperature': 42.22, 'humidity': 109.49, 'event_timestamp': 1747327890.3174708, 'error_message': '109.49 is greater than the maximum of 100'}
Inserted into DB: {'device_id': 'sensor-2', 'temperature': 42.22, 'humidity': 109.49, 'event_timestamp': 1747327890.3174708, 'error_messa